<a href="https://colab.research.google.com/github/gredy/2021Z-DataVisualizationTechniques/blob/master/Copy_of_Mini_Kaggle_Project_1_Breast_Cancer_Classification_1_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mini-kaggle-1-data-set/train.csv
/kaggle/input/mini-kaggle-1-data-set/test.csv


In [ ]:
import  numpy as np
import pandas as pd
train_data = pd.read_csv('sample_data/train.csv')

In [ ]:
train_data.head()

,id,label,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,90524101,M,17.99,20.66,117.80,991.7,0.10360,0.13040,0.120100,0.088240,...,21.080,25.41,138.10,1349.0,0.14820,0.37350,0.33010,0.19740,0.3060,0.08503
1,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.198000,0.104300,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.40000,0.16250,0.2364,0.07678
2,89346,B,9.00,14.40,56.36,246.3,0.07005,0.03116,0.003681,0.003472,...,9.699,20.07,60.90,285.5,0.09861,0.05232,0.01472,0.01389,0.2991,0.07804
3,902975,B,12.21,14.09,78.78,462.0,0.08108,0.07823,0.068390,0.025340,...,13.130,19.29,87.65,529.9,0.10260,0.24310,0.30760,0.09140,0.2677,0.08824
4,904969,B,12.34,14.95,78.29,469.1,0.08682,0.04571,0.021090,0.020540,...,13.180,16.85,84.11,533.1,0.10480,0.06744,0.04921,0.04793,0.2298,0.05974


# DATA PROCESSING

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# Load the original training dataset (which contains the test set)
df_original = pd.read_csv("sample_data/train.csv")

# Separate features and target variable for the entire dataset
df = df_original.drop(columns=["id"])  # Drop the 'id' column for feature scaling
X = df.drop(columns=["label"])
y = df["label"]  # Keep 'label' as it is ('M'/'B')

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the features for training data
X_scaled = scaler.fit_transform(X)

# Split the data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Get the test data
X_test_original = df_original[-len(X_test):]  # Get the original test data
test_ids = X_test_original['id']  # Extract the 'id' column for the original test set

# Scale the test data (excluding 'id' and 'label' columns)
X_test_scaled = scaler.transform(X_test_original.drop(columns=["id", "label"]))

# Load dataset for test.csv
test_data = pd.read_csv("sample_data/test.csv")

# Process the test dataset
X_test_final = test_data.drop(columns=["id"])  # Drop the 'id' column
X_test_final_scaled = scaler.transform(X_test_final)  # Apply the same scaling

# PERCEPTRON MODEL

* A basic neural network model that learns through updates based on misclassified points.
* Perceptron is sensitive to feature scaling, so we use StandardScaler().

In [ ]:
from sklearn.linear_model import Perceptron

# Initialize and train the Perceptron model
perceptron = Perceptron(random_state=42)
perceptron.fit(X_train, y_train)

# Get predictions for the test set
y_pred = perceptron.predict(X_test_scaled)

# Get predictions for the test data
y_pred_final = perceptron.predict(X_test_final_scaled)

# LOGISTIC REGRESSION

* LogisticRegression(random_state=42, max_iter=1000) ensures convergence for complex datasets.
* Prevents issues caused by varying feature scales.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train, y_train)

# Make predictions on validation set
y_pred = log_reg.predict(X_test)

# Get predictions for test.csv
y_pred_final = log_reg.predict(X_test_final_scaled)

# SVM

* SVC(kernel='rbf') works well for complex decision boundaries.
* SVM is sensitive to feature scales, so scaling is necessary.

In [ ]:
from sklearn.svm import SVC

# Train the SVM model
svm_model = SVC(kernel='rbf', random_state=42)  # Using RBF kernel
svm_model.fit(X_train, y_train)

# Make predictions on validation set
y_pred = svm_model.predict(X_test)

# Get predictions for test.csv
y_pred_final = svm_model.predict(X_test_final_scaled)

# DECISION TREES

* DecisionTreeClassifier(max_depth=5) prevents overfitting.
* Helps models perform better, especially with distance-based decisions.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Train the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42, max_depth=5)  # Using max_depth=5 to prevent overfitting
dt_model.fit(X_train, y_train)

# Make predictions on validation set
y_pred = dt_model.predict(X_test)

# Get predictions for test.csv
y_pred_final = dt_model.predict(X_test_final_scaled)


# KNN

* KNeighborsClassifier(n_neighbors=5) considers 5 nearest points for classification.
* KNN is sensitive to different feature scales, so StandardScaler is necessary.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Train the KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)  # Using 5 neighbors
knn_model.fit(X_train, y_train)

# Make predictions on validation set
y_pred = knn_model.predict(X_test)

# Get predictions for test.csv
y_pred_final = knn_model.predict(X_test_final_scaled)

# RANDOM FOREST

* RandomForestClassifier(n_estimators=100) creates 100 decision trees for better accuracy.
* While Random Forest isn't sensitive to scaling, keeping it consistent with previous models is good practice.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # 100 trees
rf_model.fit(X_train, y_train)

# Make predictions on validation set
y_pred = rf_model.predict(X_test)

# Get predictions for test.csv
y_pred_final = rf_model.predict(X_test_final_scaled)

# Evaluation and Submission

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluate performance on validation set
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print validation results
print(f"Validation Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)



# Load dataset for test.csv
test_data = pd.read_csv("sample_data/test.csv")

# Create submission DataFrame
submission = pd.DataFrame({
    'id': test_data['id'],  # Retain the 'id' column from test.csv
    'label': y_pred_final  # Include the predicted labels
})

# Save to a CSV file
submission.to_csv('submission.csv', index=False)

# Print confirmation
print(f"Final submission file created with {len(submission)} rows.")

Validation Accuracy: 0.9670
Classification Report:
               precision    recall  f1-score   support

           B       0.98      0.97      0.98        63
           M       0.93      0.96      0.95        28

    accuracy                           0.97        91
   macro avg       0.96      0.97      0.96        91
weighted avg       0.97      0.97      0.97        91

Final submission file created with 114 rows.


# Decision

Upon evaluation Perceptron and Logistic Regression both equally scored the best at 0.98245. I decided to choose Logistic Regression as my final submission. In most cases, Logistic Regression is the better model because it is more robust, reliable, and provides probability outputs.